In [1]:
import pandas as pd

In [2]:
data=pd.read_csv(r'C:\Users\deyar\OneDrive\Desktop\CreditCardDefaultPrediction\notebooks\data\UCI_Credit_Card.csv')

In [3]:
data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month
0,1,20000.0,2,2,1,24,2,2,-1,-1,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1
1,2,120000.0,2,2,2,26,-1,2,0,0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1
2,3,90000.0,2,2,2,34,0,0,0,0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0
3,4,50000.0,2,2,1,37,0,0,0,0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0
4,5,50000.0,1,2,1,57,-1,0,-1,0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0


In [4]:
y=data['default.payment.next.month']

In [5]:
drop_col=['ID','BILL_AMT2','BILL_AMT3','BILL_AMT4','BILL_AMT5',"default.payment.next.month"]

In [6]:
x=data.drop(drop_col,axis=1)

In [7]:
x.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
0,20000.0,2,2,1,24,2,2,-1,-1,-2,-2,3913.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0
1,120000.0,2,2,2,26,-1,2,0,0,0,2,2682.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0
2,90000.0,2,2,2,34,0,0,0,0,0,0,29239.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0
3,50000.0,2,2,1,37,0,0,0,0,0,0,46990.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0
4,50000.0,1,2,1,57,-1,0,-1,0,0,0,8617.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0


In [8]:
from sklearn.preprocessing import StandardScaler

In [9]:
s=StandardScaler()

In [10]:
x.columns

Index(['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE', 'PAY_0', 'PAY_2',
       'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6', 'BILL_AMT1', 'BILL_AMT6',
       'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3', 'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6'],
      dtype='object')

In [11]:
from sklearn.model_selection import cross_val_score

In [12]:
from sklearn.svm import SVC

In [13]:
model=SVC()

In [14]:
m=cross_val_score(model,x,y,cv=5,scoring="accuracy")
m.mean()

0.7788000000000002

In [15]:
x_sc=s.fit_transform(x)

In [16]:
m=cross_val_score(model,x_sc,y,cv=5,scoring="accuracy")
m.mean()

0.8194666666666667

In [17]:
import optuna

c:\Users\deyar\OneDrive\Desktop\CreditCardDefaultPrediction\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
def objective(trial):
  C=trial.suggest_float("C",.1,10,log=True)
  kernel=trial.suggest_categorical("kernel",["rbf"])
  gamma=trial.suggest_categorical("gamma",["scale","auto"])
  model=SVC(C=C,
          kernel=kernel,
          gamma=gamma)
  score=cross_val_score(model,x_sc,y,n_jobs=-1,cv=3)
  accuracy=score.mean()
  return accuracy

In [25]:
study=optuna.create_study(direction="maximize")

[I 2024-03-25 08:26:27,287] A new study created in memory with name: no-name-0466ba3c-f70e-404c-aff5-4f91cbd1570d


In [26]:
study.optimize(objective,n_trials=25)

[I 2024-03-25 08:26:52,877] Trial 0 finished with value: 0.8188 and parameters: {'C': 3.133670546407726, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.8188.
[I 2024-03-25 08:27:10,625] Trial 1 finished with value: 0.8176333333333333 and parameters: {'C': 0.22044889798598713, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.8188.
[I 2024-03-25 08:27:38,510] Trial 2 finished with value: 0.8177666666666666 and parameters: {'C': 6.669791165792007, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.8188.
[I 2024-03-25 08:27:55,940] Trial 3 finished with value: 0.818 and parameters: {'C': 0.36911737107602177, 'kernel': 'rbf', 'gamma': 'scale'}. Best is trial 0 with value: 0.8188.
[I 2024-03-25 08:28:13,290] Trial 4 finished with value: 0.8186666666666668 and parameters: {'C': 0.5298446203413834, 'kernel': 'rbf', 'gamma': 'auto'}. Best is trial 0 with value: 0.8188.
[I 2024-03-25 08:28:31,488] Trial 5 finished with value: 0.8188333333333334 a

In [27]:
m=study.best_trial

In [28]:
m.value

0.8192

In [29]:
m.params

{'C': 1.4958218706121758, 'kernel': 'rbf', 'gamma': 'scale'}

In [30]:
from sklearn.model_selection import train_test_split

In [31]:
x_train,x_test,y_train,y_test=train_test_split(x_sc,y,test_size=.2,random_state=12)

In [32]:
model=SVC(C=1.4958218706121758, kernel='rbf', gamma='scale')

In [33]:
model.fit(x_train,y_train)

SVC(C=1.4958218706121758)

In [34]:
p=model.predict(x_test)

In [35]:
from sklearn.metrics import accuracy_score

In [36]:
accuracy_score(y_test,p)

0.819